In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("BostonHousing.csv")
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['medv'])
y = df['medv']
cols = X.columns

scaler = StandardScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X,columns=cols)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

print('X_train:',X_train.shape)
print('y_train:',y_train.shape)
print('X_test:',X_test.shape)
print('y_test:',y_test.shape)

X_train: (404, 13)
y_train: (404,)
X_test: (102, 13)
y_test: (102,)


In [4]:
import torch
from torch import nn

class Data(torch.utils.data.Dataset):
    def __init__(self,X,y):
        super().__init__()
        self.X = torch.from_numpy(X.values)
        self.y = torch.from_numpy(y.values)
        
    def __len__(self): 
        return len(self.X)
    
    def __getitem__(self,i): 
        return (self.X[i], self.y[i])
    
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(13,32),
            nn.Sigmoid(),
            nn.Linear(32,1)
        )
        
    def forward(self, inputs):
        return self.layer(inputs)

In [6]:
NN = NeuralNetwork()
train_dataset = Data(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8)
optimizer = torch.optim.SGD(NN.parameters(), lr=1e-4)
cost = nn.MSELoss()
epochs = 1000
for epoch in range(epochs):
    total_training_loss = 0
    for i, data in enumerate(train_loader,0):
        features,target = data
        features,target = features.float(),target.float()
        target = target.reshape(-1,1)
        optimizer.zero_grad()
        pred = NN.forward(features)
        loss = cost(pred,target)
        total_training_loss += loss.item()
        loss.backward()
        optimizer.step()
    if epoch % 50 == 0:
        print(f'epoch: {epoch} - loss: {total_training_loss}')

epoch: 0 - loss: 28561.602905273438
epoch: 50 - loss: 1547.979507446289
epoch: 100 - loss: 1293.5853176116943
epoch: 150 - loss: 1203.7356700897217
epoch: 200 - loss: 1149.4461183547974
epoch: 250 - loss: 1108.542349100113
epoch: 300 - loss: 1074.9044139385223
epoch: 350 - loss: 1045.916232585907
epoch: 400 - loss: 1020.0625615119934
epoch: 450 - loss: 996.3419141769409
epoch: 500 - loss: 974.0704462528229
epoch: 550 - loss: 952.7799398899078
epoch: 600 - loss: 932.1500406265259
epoch: 650 - loss: 911.9644043445587
epoch: 700 - loss: 892.0854606628418
epoch: 750 - loss: 872.4388365745544
epoch: 800 - loss: 853.000340461731
epoch: 850 - loss: 833.7877099514008
epoch: 900 - loss: 814.8525834083557
epoch: 950 - loss: 796.2725628614426


In [8]:
preds = NN.forward(torch.from_numpy(X_test.values).float())


In [10]:
preds = preds.detach().numpy().reshape(-1,)
result = pd.DataFrame(
    {
        'Actual':y_test.values,
        'Prediction':preds
    }
)
result

,Actual,Prediction
0,15.4,14.053814
1,48.3,40.507362
2,32.0,33.389210
3,17.2,14.860342
4,25.0,28.358135
...,...,...
97,25.3,25.549902
98,26.2,25.701357
99,17.2,12.182201
100,10.2,13.634985
